# Setup LangSmith API
Retrievals can be traced here for easier debugging.

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = ' tryurbest-2-guess-what-it_i5* '

# Graph States

In [ ]:
from dataclasses import dataclass, field
from typing import Optional, List, Dict, Any

@dataclass
class VetAssistantState:
    text_query: str
    image: Optional[Any] = None  # e.g., file path, URL, or bytes
    image_summary: Optional[str] = None
    triage_result: Optional[str] = None
    conversation_history: List[Dict[str, str]] = field(default_factory=list)  # [{'role': 'user', 'content': ...}, ...]
    clarification_needed: bool = False
    clarification_question: Optional[str] = None
    clarification_answer: Optional[str] = None
    retrieved_docs: List[Dict[str, Any]] = field(default_factory=list)
    final_answer: Optional[str] = None
    citations: List[str] = field(default_factory=list)
    web_search_results: List[Dict[str, Any]] = field(default_factory=list)
    path_taken: List[str] = field(default_factory=list)
    symptom_list: List[str] = field(default_factory=list)
    follow_up_count: int = 0
    error: Optional[str] = None
    awaiting_user_feedback: bool = False
    current_probe: Optional[str] = None  
    probe_result: Optional[str] = None  

<h1> Graph Nodes

<h3>1. Triage Router Node</h3>

Classifies user intent: 'emergency', 'symptom_check', or 'general_qa'.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama


triage_prompt = PromptTemplate(
    template=
    "You are a veterinary assistant triage system. "
    "Classify the following user input into one of three categories: "
    "'emergency', 'symptom_check', or 'general_qa'.\n\n"
    "User text input: {text_query}\n"
    "{image_block}"
    "\nRespond strictly with only one word: emergency, symptom_check, or general_qa."
    ,
    input_variables=["text_query", "image_block"]
)

triage_router = triage_prompt | ChatOllama(model="llama3.2:3b", temperature=0.0) | StrOutputParser()

